In [1]:
import pandas as pd
import sqlite3
from transformers import (
    RobertaTokenizer,
    T5ForConditionalGeneration,
    DataCollatorForLanguageModeling
)
from modules.models import CodeT5
from modules.datasets import CodeT5Dataset
from modules.TrainConfig import init_logger, init_checkpoint, Trainer
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

LOG_PATH = 'logs',
MODEL_DIR = 'Model Checkpoints/CodeT5JS'
CPKT_PATH = 'Model Checkpoints/CodeT5JS'
VERSION = 0

/home/disras/miniconda3/envs/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print('1. Loading Samples..')
con = sqlite3.connect('commitpack-datasets.db')

with open('query.sql', 'r') as f:
    query = f.read()
ds_df = pd.read_sql_query(query,con).set_index('index')[:12]
old_codes = ds_df['old_contents'].tolist()
new_codes = ds_df['new_contents'].tolist()

TRAIN_old, VAL_old, TRAIN_new, VAL_new = train_test_split(old_codes, new_codes, test_size=0.15, random_state=42)

1. Loading Samples..


In [3]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')

TRAIN_encodings = tokenizer(
    TRAIN_old,
    max_length=512,
    pad_to_max_length=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True
)

VAL_encodings = tokenizer(
    VAL_old,
    max_length=512,
    pad_to_max_length=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True
)

TRAIN_decodings = tokenizer(
    TRAIN_new,
    max_length=512,
    pad_to_max_length=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True
)

VAL_decodings = tokenizer(
    VAL_new,
    max_length=512,
    pad_to_max_length=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True
)

KeyboardInterrupt: 

In [ ]:
model = CodeT5()

In [ ]:
model.model.train()

TRAIN_dataset = CodeT5Dataset(TRAIN_encodings, TRAIN_decodings)
VAL_dataset = CodeT5Dataset(VAL_encodings, VAL_decodings)
dataloader = DataLoader(TRAIN_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(VAL_dataset, batch_size=1)

logger = init_logger(LOG_PATH, MODEL_DIR, VERSION)
checkpoint = init_checkpoint(CPKT_PATH, MODEL_DIR, VERSION)
trainer = Trainer(checkpoint,logger,debug=True)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/disras/miniconda3/envs/thesis/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1584: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).


In [ ]:
trainer.fit(
    model,
    train_dataloaders=dataloader,
    val_dataloaders=val_dataloader
)


  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.528   Total estimated model params size (MB)


Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] 

ValueError: too many values to unpack (expected 2)